In [44]:
from transformers import AutoTokenizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from huggingface_hub import notebook_login

from datasets import *
import numpy as np
import re
import random


In [62]:

dataset = load_from_disk('AR-dotted-mediumPlus-arrow')

In [63]:
tokenizer = AutoTokenizer.from_pretrained("AR-dotted-tokenizer")


In [47]:
test_sentance = "وجميل [MASK] الطقس"
test_sentance

'وجميل [MASK] الطقس'

In [48]:
new = allVariation(parse_text("معتدل")[1])
print(new)
print("Length:", len(new))
print("Duplicates:", len(new) != len(set(new)))


NameError: name 'allVariation' is not defined

In [49]:
def mask_random_word(input_string, seed=None):
    words = input_string.split()
    print(words)
    if seed is not None:
        random.seed(seed)

    if words:
        index = random.randint(0, len(words) - 1)
        word_to_replace = words[index]
        words[index] = "[MASK]"
        modified_string = ' '.join(words)
        return modified_string, word_to_replace
    else:
        return input_string, None

In [50]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean', 'dotless'],
        num_rows: 2980842
    })
})

In [51]:

#test_sentance = "الطقس معتدل وجميل"
test_sentance = dataset["train"][0]['clean']
seed = None
#seed = 1
masked_sentance, target = mask_random_word(test_sentance, seed=seed)
variations = allVariation(parse_text(target)[1])
print("Original Sentance:", test_sentance)
print("Masked Sentance:", masked_sentance)
print("Target:", target)
print("Variations:", variations)

print()
print()
print()

# sanity check to make sure the way arabic is messing up with 
# english in text display isn't actually how it is.
masked_Array = masked_sentance.split()
for i in range(len(masked_Array)):
    print(masked_Array[i])
print()
print()
print()

inputs_c = tokenizer(masked_sentance)
print(inputs_c)
print(tokenizer.convert_ids_to_tokens(inputs_c["input_ids"]))
print(masked_sentance)

['زكريا', 'محيي', 'الدين', 'يحيى', 'بن', 'شرف', 'النووي', 'المتوفى', 'ه', 'ج', 'ص']


NameError: name 'allVariation' is not defined

## New Dataset creation, MASKED sentance, list of possible targets, and label (target)

In [52]:
# Replaces multiple spaces, or whatever char designated by the perameter into one char. (helper function)
def replace(string, char):
    while char+char in string:
        string = string.replace(char+char, char)

    return string

def reverse_dict_with_duplicates(input_dict):
    reversed_dict = {}
    
    for key, value in input_dict.items():
        if value not in reversed_dict:
            reversed_dict[value] = [key]
        else:
            reversed_dict[value].append(key)
    
    return reversed_dict

letter_dict = {
    "ا": "ا",
    "أ": "ا",
    "إ": "ا",
    "آ": "ا",
    "ب": "ب",
    "ت": "ب",
    "ث": "ب",
    "ج": "ح",
    "ح": "ح",
    "خ": "ح",
    "د": "د",
    "ذ": "د",
    "ر": "ر",
    "ز": "ر",
    "س": "س",
    "ش": "س",
    "ص": "ص",
    "ض": "ص",
    "ط": "ط",
    "ظ": "ط",
    "ع": "ع",
    "غ": "ع",
    "ف": "ف",
    "ق": "ف",
    "ك": "ك",
    "ل": "ل",
    "م": "م",
    "ن": "ن",
    "و": "و",
    "ؤ": "و",
    "ه": "ه",
    "ة": "ه",
    "ي": "ى",
    "ى": "ى",
    "ئ": "ى",
    " ": "_",
    
}
letter_set = set(letter_dict.keys())
iLetters = reverse_dict_with_duplicates(letter_dict)

In [79]:
def uMap_parse_dotless_text(text):
  dotless_chars = [letter_dict[char] for char in text]
  dotless_string = ''.join(dotless_chars)
        
  dotless_string = replace(dotless_string, '_')
    
  return dotless_string

In [119]:
def uMap_allVariation(root, i = 0):
    tWord = []
    word = [a for a in root]
    char = word[i]
    for dot in iLetters[char]:
        word[i] = dot
        vWord = ''.join(word)
        if i != len(word) - 1:
            tWord.extend(uMap_allVariation(vWord, i+1))
        else:
            #if vWord in dictionary:
            tWord.append(vWord)
    return tWord

In [175]:
import time


In [215]:
n = 10000
prediction_n = each_n*n

print("predicted time for", n, "examples:", prediction_n, "seconds")
print("\n\n\n\n")
t0 = time.time()
for i in range(n): uMap_allVariation(uMap_parse_dotless_text("زكريازكريا"))

t1 = time.time()


total_n = (t1-t0)
each_n = total_n/n
print("average per example:", each_n, "seconds")
print("--------------------------------------------------------------------------")
print("predicted time for", n, "examples:", prediction_n, "seconds")
print("actual time for", n, "examples:   ", total_n, "seconds")
print("accuracy of prediction:", (abs(total_n - prediction_n)/total_n)*100, "%")


predicted time for 1000 examples: 0.798271894454956 seconds





average per example: 0.0007939298152923584 seconds
--------------------------------------------------------------------------
predicted time for 1000 examples: 0.798271894454956 seconds
actual time for 1000 examples:    0.7939298152923584 seconds
accuracy of prediction: 0.546909698938403 %


In [209]:
#def helper_map_allVariation(example):
 #   strings = []
 #   print(example)
  #  for text in example:
   #     print(text)
    #    strings.append(map_allVariation(text))

In [100]:
def map_mask_random_word(example):
    masked = []
    options = []
    targets = []

    for text in example["clean"]:
        words = text.split()
        
        index = random.randint(0, len(words) - 1)  # Generate a random index for each sentence

        for i, word in enumerate(words):
            if i == index:
                word_to_replace = words[i]
                variations = uMap_allVariation(uMap_parse_dotless_text(word_to_replace))
                words[i] = "[MASK]"
                modified_string = ' '.join(words)
                masked.append(modified_string)
                options.append(variations)
                targets.append(word_to_replace)
    
    return {"Masked": masked, "Options": options, "Target": targets}

In [109]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean'],
        num_rows: 2980842
    })
})

In [118]:
#n_dataset= dataset
n_dataset = dataset['train'].select(range(100000)) #Trim to number of examples #previously 100,000
print(n_dataset, '\n')
print(n_dataset[0])

Dataset({
    features: ['clean'],
    num_rows: 100000
}) 

{'clean': 'زكريا محيي الدين يحيى بن شرف النووي المتوفى ه ج ص'}


In [117]:
m_dataset = n_dataset.map(map_mask_random_word, batched= True, remove_columns="clean") #Apply the function (convert dotted to dotless)


Map:   0%|          | 0/100001 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [105]:
print(m_dataset)
m_dataset[0]


Dataset({
    features: ['Masked', 'Options', 'Target'],
    num_rows: 10000
})


{'Masked': 'زكريا محيي الدين [MASK] بن شرف النووي المتوفى ه ج ص',
 'Options': ['يجيي',
  'يجيى',
  'يجيئ',
  'يجىي',
  'يجىى',
  'يجىئ',
  'يجئي',
  'يجئى',
  'يجئئ',
  'يحيي',
  'يحيى',
  'يحيئ',
  'يحىي',
  'يحىى',
  'يحىئ',
  'يحئي',
  'يحئى',
  'يحئئ',
  'يخيي',
  'يخيى',
  'يخيئ',
  'يخىي',
  'يخىى',
  'يخىئ',
  'يخئي',
  'يخئى',
  'يخئئ',
  'ىجيي',
  'ىجيى',
  'ىجيئ',
  'ىجىي',
  'ىجىى',
  'ىجىئ',
  'ىجئي',
  'ىجئى',
  'ىجئئ',
  'ىحيي',
  'ىحيى',
  'ىحيئ',
  'ىحىي',
  'ىحىى',
  'ىحىئ',
  'ىحئي',
  'ىحئى',
  'ىحئئ',
  'ىخيي',
  'ىخيى',
  'ىخيئ',
  'ىخىي',
  'ىخىى',
  'ىخىئ',
  'ىخئي',
  'ىخئى',
  'ىخئئ',
  'ئجيي',
  'ئجيى',
  'ئجيئ',
  'ئجىي',
  'ئجىى',
  'ئجىئ',
  'ئجئي',
  'ئجئى',
  'ئجئئ',
  'ئحيي',
  'ئحيى',
  'ئحيئ',
  'ئحىي',
  'ئحىى',
  'ئحىئ',
  'ئحئي',
  'ئحئى',
  'ئحئئ',
  'ئخيي',
  'ئخيى',
  'ئخيئ',
  'ئخىي',
  'ئخىى',
  'ئخىئ',
  'ئخئي',
  'ئخئى',
  'ئخئئ'],
 'Target': 'يحيى'}

In [106]:
dataset_resplit = m_dataset.train_test_split(train_size= 0.999999999) #create 80/20 split into train and test datasets
del dataset_resplit['test'] # no longer using a test set
dataset_resplit


DatasetDict({
    train: Dataset({
        features: ['Masked', 'Options', 'Target'],
        num_rows: 9999
    })
})